### 3/ Affichage des maps de top 20 et top 5 de destination

In [2]:
import pandas as pd
import plotly.graph_objects as go

Dans la continuité du notebook n°2, on utilise le dataframe produit avec la météo moyenne en journée de chacune des 35 destinations, pour une petite 'preuve de concept' de la visualisation utilisateur recherchée.
Il nous faut produire deux maps (un top 20 et un top 5) des meilleures destinations météo en France au jour J.

On décide de permettre à l'utilisateur de visualiser sur la carte de France ces meilleurs destinations, par défault, grâce au paramètre "feels_like' (bon ressenti global) moyen que nous avions récupéré. Mais nous allons aussi permettre à l'utilisateur de faire varier le filtre de la map en fonction de critères plus personnnels : température, vitesse du vent, etc...

Récupération de notre dataframe :

In [3]:
data = pd.read_csv('results/result_cities_with_coord_and_weather.csv')

In [4]:
data.head()

,Unnamed: 0,city,lat,lon,feels_avg,temp_avg,humid_avg,wind_avg,cloud_descr_avg,pop_avg,start_date,end_date
0,0,Mont Saint Michel,48.635954,-1.511460,18.565,18.755,72.0,4.640,broken clouds,0.0,2022-10-03 11:19:15.796598,2022-10-08 11:19:15.796598
1,1,St Malo,48.649518,-2.026041,18.335,18.485,74.5,4.435,broken clouds,0.0,2022-10-03 11:19:15.796598,2022-10-08 11:19:15.796598
2,2,Bayeux,49.276462,-0.702474,16.285,16.720,70.5,3.295,overcast clouds,0.0,2022-10-03 11:19:15.796598,2022-10-08 11:19:15.796598
3,3,Le Havre,49.493898,0.107973,14.750,15.035,82.5,2.860,broken clouds,0.0,2022-10-03 11:19:15.796598,2022-10-08 11:19:15.796598
4,4,Rouen,49.440459,1.093966,14.485,15.055,71.5,2.975,broken clouds,0.0,2022-10-03 11:19:15.796598,2022-10-08 11:19:15.796598


Quelle est cette première colonne doublon de l'index ? Aïe, cela est dû à l'export puis import en csv...
On supprime la colonne inutile :

In [5]:
del data['Unnamed: 0']

In [7]:
data.dtypes

city                object
lat                float64
lon                float64
feels_avg          float64
temp_avg           float64
humid_avg          float64
wind_avg           float64
cloud_descr_avg     object
pop_avg            float64
start_date          object
end_date            object
dtype: object

Pour la même raison (passage en csv), visiblement, nos dates sont repassées en string...
Il faut les reconvertir une nouvelle fois :

In [9]:
import datetime as dt

data['start_date']=data['start_date'].map(lambda s: dt.datetime.strptime(s.replace(".796598",""),"%Y-%m-%d %H:%M:%S"))
data['end_date']=data['end_date'].map(lambda s: dt.datetime.strptime(s.replace(".796598",""),"%Y-%m-%d %H:%M:%S"))

In [10]:
data.dtypes

city                       object
lat                       float64
lon                       float64
feels_avg                 float64
temp_avg                  float64
humid_avg                 float64
wind_avg                  float64
cloud_descr_avg            object
pop_avg                   float64
start_date         datetime64[ns]
end_date           datetime64[ns]
dtype: object

On doit aussi trouver un moyen d'ordonner la variable 'cloud_descr_avg' qui est pour l'instant un string. Cette donnée catégorielle peut être facilement ordonnée (taux de couverture nuageux du ciel... + autres évènements) du plus propice au moins propice, ou l'inverse...
Mais avant de faire ce travail de conversion, il faut savoir à quelles catégories on a affaire. Sur notre dataset on a 4 catégories, visiblement :

In [335]:
data.groupby('cloud_descr_avg').count()

,city,lat,lon,feels_avg,temp_avg,humid_avg,wind_avg,pop_avg,start_date,end_date
cloud_descr_avg,,,,,,,,,,
broken clouds,11,11,11,11,11,11,11,11,11,11
clear sky,17,17,17,17,17,17,17,17,17,17
overcast clouds,6,6,6,6,6,6,6,6,6,6
scattered clouds,1,1,1,1,1,1,1,1,1,1


Ok, mais sont-ce les seules ?... En allant sur la doc de l'API Open Weather Map, on voit que d'autres catégories sont possibles (brouillard, orages, neige, pluie...)
Il est possible que nous relancions ce programme à un autre moment et que donc ces catégories apparaissent.
On fait donc le travail de conversion avec toutes les catégories de 'beau ou mauvais ciel' en attribuant une note sur 10 au ciel, 10 étant la plus pire des notes ! (pour des raisons pratiques d'affichage plus loin). Cet ordre reste assez arbitraire, mais nous nosu plaçons à la place du touriste.
Exemple : qu'est-ce qui est pire en vacance ? Faire face à des orages ou du brouillard ? Sans doute le brouillard que l'orage qui reste généralement plus passager. Le brouillard empêche de voir les paysages...

In [11]:
data['cloud_order']=data['cloud_descr_avg'].map({
                                                'mist':10,
                                                'thunderstorm':9,
                                                'snow':8,
                                                'rain':7,
                                                'shower rain':6,
                                                'overcast clouds':5,
                                                'broken clouds':4,
                                                'scattered clouds':3,
                                                'few clouds':2,
                                                'clear sky':1})

In [12]:
data.columns

Index(['city', 'lat', 'lon', 'feels_avg', 'temp_avg', 'humid_avg', 'wind_avg',
       'cloud_descr_avg', 'pop_avg', 'start_date', 'end_date', 'cloud_order'],
      dtype='object')

Pour des raisons pratiques d'affichage, on arrondit ici la donnée 'probabilité de précipitations' à 3 décimales :

In [338]:
data['pop_avg']=round(data['pop_avg'],3)

Notre programme d'affichage des maps va faire appelle un certain nombre de fois la fonction ci-dessous qui permet d'automatiser la modification d'un dataframe donné en fonction d'un paramètre météo donné par l'utilisateur comme 'temp', 'humidity', etc... La fonction ci-dessous créé donc le top 20 ou top 5 (ou top x) recherché en fonction de ce paramètre, du dataframe, mais aussi du nombre de destinations retenues sur le 'podium' ainsi constitué :

In [13]:
def order_by(dataset,top_nb,tag='feels_avg'):
    if tag not in dataset.columns:
        return "Error : please enter a good parameter to order this dataset..."
    elif top_nb > len(dataset):
        return "Error : the top_number is most than data size..."
    else :
        if tag=='humid_avg' or tag=='pop_avg' or tag=='wind_avg' or tag=='cloud_order':
            # pour ces quatre paramètres on ordonne dans le sens descendant
            # exemple : plus il y a de vent, moins c'est agréable pour le tourisme
            return data.sort_values(tag,ascending=True).iloc[:top_nb,:]
        else:
            # pour les autres paramètres (good_feels et temp), dans le sens ascendant
            return data.sort_values(tag,ascending=False).iloc[:top_nb,:]

On prépare ici deux variables strings pour pouvoir afficher les dates de début et de fin de séjour (de prévisions) dans les maps :

In [14]:
day_start=min(data['start_date']).strftime('%d/%m')
day_end=min(data['end_date']).strftime('%d/%m')

Création de la fonction de visualisation de map avec Plotly graph_objects.

On donne en entrée le dataset et le top que l'on veut (ici, 5 ou 20) et la fonction fait le travail et nous retourne l'affichage d'une Scattermapbox avec, en plus, un système de filtre pour l'utilisateur, pour répondre au besoin décrit précedemment.

Dans la fonction, on constitue en préliminaire, selon les variables d'entrées, une liste "labels" qui va stocker, dans plusieurs dictionnaires différents, selon les paramètres météo différents, nos données brutes :
- le dataset ordonné selon le paramètre météo
- le nom du paramètre météo
- le titre à donner à l'échelle de la map (pour l'affichage)
- la charte graphique utilisée pour l'affichage de la map, pour ce paramètre (on choisit des chartes graphiques évocatrices pour chaque paramètre météo)
- la visibilité initiale de la map (par défault, seule la map de 'bonne météo globale' (indicateur 'feels_like' de OWM) sera visible). paramètre qui évoluera ensuite avec le jeu des bouttons

In [23]:
from tkinter import font
from unittest import skip
from matplotlib import markers

def map_of_best_dest(data,top_nb):

    data = order_by(data,len(data))
    # petite précaution : on ordonne avant tout le dataframe selon le
    # paramètre par défault 'feels_avg', car on risque sur nos différents
    # autres 'podiums' (pour 'temp', 'wind', etc...) d'avoir des égalités de scores
    # donc ces égalités seront arbitrés par ce super-paramètre 'feels_avg'

   

    # stockage des différents données dans différents dictionnaire
    # (1 par paramètre météo) :

    labels=[{'data':order_by(data,top_nb),
        'param':'feels_avg',
        'title':"Echelle de bonne meteo globale (OWM)",
        'colorbar':'purpor',
        'init_visible':True},
        {'data':order_by(data,top_nb,'temp_avg'),
        'param':'temp_avg',
        'title':"Echelle de températures (°C)",
        'colorbar':'oranges',
        'init_visible':False},
        {'data':order_by(data,top_nb,'wind_avg'),
        'param':'wind_avg',
        'title':"Echelle de vent faible (noeuds)",
        'colorbar':'tealgrn',
        'init_visible':False},
        {'data':order_by(data,top_nb,'humid_avg'),
        'param':'humid_avg',
        'title':"Echelle de taux d'humidité relative (%)",
        'colorbar':'blugrn',
        'init_visible':False},
        {'data':order_by(data,top_nb,'cloud_order'),
        'param':'cloud_order',
        'title':"Echelle de ciel encombré (sur 10)",
        'colorbar':'blues',
        'init_visible':False},
        {'data':order_by(data,top_nb,'pop_avg'),
        'param':'pop_avg',
        'title':"Echelle de probabilité de precipitations",
        'colorbar':'mint',
        'init_visible':False}]

    fig = go.Figure()

    # Pour chaque paramètre météo (chaque dictionaire de 'labels')
    # on ajoute une trace de Scattermapbox

    for label in labels:
        fig.add_trace(
            go.Scattermapbox(
                lat=label.get('data')['lat'],
                lon=label.get('data')['lon'],
                hovertext=label.get('data')['city'],
                hoverinfo='text',
                mode='markers',
                marker=dict(
                    size=16,
                    cmax=label.get('data')[label.get('param')].max(),
                    cmin=label.get('data')[label.get('param')].min(),
                    color=label.get('data')[label.get('param')],
                    colorscale=label.get('colorbar'),
                    colorbar=dict(
                        title=dict(
                            side='right',
                            text=label.get('title'),
                            font=dict(
                                color='purple',
                                size=16,
                                family='Arial'
                                )
                            ),
                    bgcolor='aliceblue',
                    x=1.08,
                    y=0.5,
                    len=1.1)
                    ),
                visible=label.get('init_visible')
                )
        )

    # Paramètres globaux de layout:

    fig.update_layout(mapbox_style='stamen-terrain',
                    title=dict(
                        text=f'Le Top {top_nb} des destinations météo en France ! Du {day_start} au {day_end}',
                        font=dict(
                            color='purple',
                            size=24,
                            family='Open Sans'
                        )
                    ))

    # Paramètres supplémentaires internes à la dynamique mapbox :

    fig.update_mapboxes(
        bearing=0,
        center=dict(
            lat=46,
            lon=4
        ),
        pitch=0,
        zoom=3.7)

    # Gestion des boutons pour l'utilisateur des maps :

    fig.update_layout(
        updatemenus = [go.layout.Updatemenu(
            active = 0,
            buttons = [
                    go.layout.updatemenu.Button(
                        label = "Par météo globale (en journée)",
                        method = "update",
                        args = [{"visible" : [True, False, False, False, False, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Par températures moyennes (en journée)",
                            method = "update",
                            args = [{"visible" : [False, True, False, False, False, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Par vitesse de vent moyen (en journée)",
                            method = "update",
                            args = [{"visible" : [False, False, True, False, False, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Par taux d'humidité relative moyenne (en journée)",
                            method = "update",
                            args = [{"visible" : [False, False, False, True, False, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Par encombrement du ciel moyen (en journée)",
                            method = "update",
                            args = [{"visible" : [False, False, False, False, True, False]}]),
                    go.layout.updatemenu.Button(
                            label = "Par probabilité moyenne de pluie (en journée)",
                            method = "update",
                            args = [{"visible" : [False, False, False, False, False, True]}])
                ]
            )]
        )

    fig.show()

Allez, on affiche notre top 20...

In [24]:
map_of_best_dest(data,20)

Et maintenant notre top 5...

In [25]:
map_of_best_dest(data,5)

L'équipe de développement front-end affinera tout cela !... 

Il manque aussi à l'utilisateur quelques infos pour savoir comment a été calculé chaque paramètre. Mais cela donne une bonne idée de la visualisation possible.
(la direction décidera si on laisse à l'utilisateur la possibilité d'agir sur les boutons (filtres) ou s'il vaut mieux réduire l'affichage de la carte sur le site à son strict minimun ;) mais nous, on aime bien l'idée ! car cela permet une intéraction et intuitivement, l'utlisateur a l'impression qu'on lui 'prescrit' des destinations de 'bonne météo globale' en fonction de divers sous-paramètres qu'il peut visualiser aussi... Cela donne du crédit au paramètre initial 'bonne météo' très arbitraire... Même si les deux ne sont pas corrélés (on rappelle que la variable 'feels-like' est née d'une autre formule particulière interne à l'API Open Weather Map, qu'il faudrait également afficher sur le site par souci d'honnêteté scientifique )